In [67]:
#importing the packages

#libraries for NLP
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer =LancasterStemmer()

#libraries needed for tensor flow

import tensorflow as tf
import numpy as np
!pip install tflearn
import tflearn
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [68]:
from google.colab import files
files.upload()

Saving intents.json to intents (1).json


{'intents.json': b'{"intents": [\r\n        {"tag": "greeting",\r\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"],\r\n         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "goodbye",\r\n         "patterns": ["Bye", "See you later", "Goodbye"],\r\n         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\r\n        },\r\n        {"tag": "thanks",\r\n         "patterns": ["Thanks", "Thank you", "That\'s helpful"],\r\n         "responses": ["Happy to help!", "Any time!", "My pleasure"]\r\n        },\r\n        {"tag": "hours",\r\n         "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\r\n         "responses": ["We\'re open every day 9am-9pm", "Our hours are 9am-9pm every day"]\r\n        },\r\n        {"tag": "location",\r\n         "patterns": 

In [69]:
#loading the json file
with open('intents.json') as file:
  intents = json.load(file)

In [70]:
intents

{'intents': [{'context_set': '',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day'],
   'tag': 'hours'},
  {'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of London Alley and Bridge A

In [71]:
words=[]
classes=[]
documents=[]
#ignore the certain punctuation words
ignore=['?']
#loop through each sentences in the intents patterns
for intent in intents['intents']:
  for pattern in intent['patterns']:
    #tokenizing the each word
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    #add words to documets
    documents.append((w,intent['tag']))
    #adding the tags to our classes
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [72]:
#performing stemming and removing the duplications
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

#removing the duplicate classes
classes = sorted(list(set(classes)))

print(len(documents),'documnets')
print(len(classes),'classes',classes)
print(len(words),'words',words)

31 documnets
9 classes ['deliveryoption', 'goodbye', 'greeting', 'hours', 'location', 'menu', 'payments', 'thanks', 'todaysmenu']
57 words ["'s", 'acceiv', 'address', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'cours', 'credit', 'day', 'del', 'delicy', 'delivery', 'dish', 'do', 'food', 'for', 'from', 'good', 'goodby', 'hello', 'help', 'hi', 'hom', 'hour', 'how', 'is', 'lat', 'loc', 'main', 'mastercard', 'me', 'menu', 'most', 'on', 'op', 'opt', 'provid', 'resta', 'see', 'serv', 'situ', 'spec', 'tak', 'tel', 'thank', 'that', 'the', 'ther', 'today', 'what', 'when', 'wher', 'yo', 'you']


In [73]:
#creating the training data
training=[]
output=[]
#creating an empty array for output
output_empty = [0]*len(classes)

#creating the training set and bag of words for each sentences
for doc in documents:
  bag=[]
  pattern_words=doc[0]
  #stemming
  pattern_words = [stemmer.stem(word.lower())for word in pattern_words]
  #creating the bag of words 
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1
  training.append([bag,output_row])

#shuffling the featires and converting into array
random.shuffle(training)
training = np.array(training)

#creating the training lists
train_x = list(training[:,0]) #features
train_y = list(training[:,1])  #labels

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [74]:
#resetting the underliying the graph
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [75]:
#building the neural network
net = tflearn.input_data(shape=[None,len(train_x[0])])
net = tflearn.fully_connected(net,10)
net = tflearn.fully_connected(net,10)
net = tflearn.fully_connected(net,len(train_y[0]),activation='softmax')
net = tflearn.regression(net)

#defiining the model and building the tensor board

model =tflearn.DNN(net,tensorboard_dir='tflearn_logs')

#training the model
model.fit(train_x,train_y,n_epoch=1000,batch_size=8,show_metric=True)


Training Step: 3999  | total loss: 1.36918 | time: 0.018s
| Adam | epoch: 1000 | loss: 1.36918 - acc: 0.9080 -- iter: 24/31
Training Step: 4000  | total loss: 1.23315 | time: 0.022s
| Adam | epoch: 1000 | loss: 1.23315 - acc: 0.9172 -- iter: 31/31
--


In [76]:
model.save('model.tflearn')

INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [77]:
import pickle
pickle.dump({'words':words,'classes':classes,'train_x':train_x,'train_y':train_y},open('training_data','wb'))

In [78]:
#restoring the saved model files
data = pickle.load(open('training_data','rb'))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [79]:
intents = json.load(open('intents.json'))

In [80]:
#loading the training file 
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [81]:
#processing the intents
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  #stemming each word
  #print(sentence_words)
  sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
  #print(sentence_words)
  return sentence_words

#returning the bag of words array abd creating the labels whether it conatins it or not
def bow(sentence,word,show_details=False):
  sentence_words = clean_up_sentence(sentence)
  #generating the bag of words
  bag = [0]*len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w ==s:
        bag[i] = 1
        if show_details:
          print('found in bag: %s' %w)
  return (np.array(bag))

In [82]:
#buliding the response classifier
ERROR_THRESHOLD=0.38 #ignoring the words less that 0.38 probability
def classify(sentence):
  results = model.predict([bow(sentence,words)])[0]
  #filtering the prediction less than the treshold
  results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
      return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
  return return_list

def response(sentence,userID='123',show_details=False):
  results = classify(sentence)
  #if contains the classification then we find the matching intent
  if results:
    while results:
      for i in intents['intents']:
        if i['tag'] == results[0][0]:
          return print(random.choice(i['responses']))
      results.pop(0)
      

In [83]:
#testing for getting most likely word based on prbability
classify('What are your hours of operation')

[('hours', 0.99841475)]

In [84]:
response('What are your hours of operation')

We're open every day 9am-9pm


In [85]:
classify('What is menu')

[('menu', 0.97014356)]

In [86]:
response('What is menu')

You can visit www.mymenu.com for menu options


In [87]:
classify('That is helpful')

[('thanks', 0.98033834)]

In [88]:
response('That is helpful')

My pleasure


In [89]:
#Adding some context to the conversation i.e. Contexualization for altering question and intents etc.
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [90]:
response('Can you please let me know the delivery options?')


We have home delivery options through UBER Eats and Zomato


In [91]:
response('What is menu for today?')


Our speciality for today is Chicken Tikka


In [92]:
context

{'123': 'food'}